In [ ]:
import openai
import pandas as pd
import time
import requests
import json
import jsonlines
import re

sk = "YOUR API KEY"
openai.api_key = sk

def chat(input_data, model="gpt-3.5-turbo", temperature=0.8):
    
    nmessages = [{"role": "user", "content": input_data, "temperature": temperature}]

    while (1):
        try:
            response = openai.ChatCompletion.create(          
                model=model,
                messages=nmessages
            )
            resmessage = response['choices'][0]['message']['content']
            break
        except:
            time.sleep(10)
            
    return  resmessage

In [ ]:
processed_tuples = []
with open('./results/business/GPT_business_with_retrieved_tuples_by_monoT5.jsonl', 'r') as f:
    for line in f:
        line = json.loads(line)
        tuple_id = int(line['tuple_id'])
        processed_tuples.append(tuple_id)

print(len(processed_tuples))

In [ ]:
import json
import jsonlines
import pandas as pd
import ast

tuples = {}
template = '''What's the most likely value for the [TO-FILL] cell in the table below? Please respond using JSON: {answer_format}, the key is attribute name of each [TO-FILL], value is the predicted value for each [TO-FILL].\n'''

query_ids = []
with open('/Users/yichendezaizai/Data_Imputation/data/business/annotated_data/queries.tsv', 'r') as f:
    for line in f:
        line = line.strip()
        qid = int(line[:line.index('\t')])
        query_ids.append(qid)


# 读取表格数据tv_ret_1.csv
tableData = []
ground_truth = []
missing_columns = ['CITY', 'ZIP CODE']
columns_to_keep = ['ACCOUNT NUMBER', 'SITE NUMBER', 'NAME', 'CITY', 'STATE', 'ZIP CODE', 'LICENSE DESCRIPTION', 'APPLICATION TYPE']

df = pd.read_csv('/Users/yichendezaizai/Data_Imputation/data/business/Business_Licenses_-_Pharmaceutical_Representative.csv', usecols=columns_to_keep)


for index, row in df.iterrows():
    
    if index > 4242:
        break

    if index not in query_ids or index in processed_tuples:
        continue

    input_data = template + '[caption]: Business_Licenses_-_Pharmaceutical_Representative ' + '\n'
    for col in df.columns:
        input_data += '|' + col
    input_data += '|\n'
    
    answer_format = '{'
    missing_pos = []
    for col_id, col in enumerate(df.columns):
        # if col in missing_columns and not pd.isnull(row[col]):
        if col in missing_columns:
            cell_value = '[TO-FILL]'
            answer_format += col + ": " + '""' + ", "
            missing_pos.append(col_id)
        else:
            cell_value = row[col]
        input_data += '|' + str(cell_value)
        
    input_data += '|\n'
    answer_format = answer_format[:-2] + '}'
    input_data = input_data.format(answer_format=answer_format)

    print("---------------------------------------------------")
    print(f"Input: \n{input_data}")

    output = chat(input_data, model="gpt-3.5-turbo", temperature=0.3)

    print(f"Output: \n{output}")
            
    imputed_data = ast.literal_eval(output)

    
    fout = jsonlines.open('./results/business/GPT_business_wo_evidence.jsonl', 'a')
    fout.write({'tuple_id':index, 'input': input_data, 'output':output, 'imputed_data': imputed_data})
    fout.close()



## Imputed Data with retrieved tables

In [ ]:
top_K = 20

In [ ]:
from collections import defaultdict

all_scores = defaultdict(dict)
with open('/Users/yichendezaizai/Data_Imputation/retrieval_results/rerank_results/business_epoch40.test.tsv', 'r') as f:
    for line in f:
        qid, docid, rank, score = line.strip().split('\t')
        # qid, docid, score = line.strip().split('\t')
        score = float(score)
        all_scores[int(qid)][int(docid)] = score

qq = list(all_scores.keys())

# topK_pids
topK_results = {}
for qid in qq:
    score_list = sorted(list(all_scores[qid].items()), key=lambda x: x[1], reverse=True)
    for rank, (docid, score) in enumerate(score_list):
        if rank >= top_K:
            continue
        if qid not in topK_results:
            topK_results[qid] = []
        topK_results[qid].append(docid)


In [ ]:
topK_results[2256]

In [ ]:
collection = {}
with open('/Users/yichendezaizai/Data_Imputation/data/business/annotated_data/collection.tsv', 'r') as f:
    for line in f:
        line = line.strip()
        qid, query = line[:line.find('\t')], line[line.find('\t')+1:]
        collection[int(qid)] = query

In [ ]:
collection[457516]

In [ ]:
processed_tuples = []
#with open('./results/business/GPT_business_with_retrieved_tuples_by_monoT5.jsonl', 'r') as f:
with open('/Users/yichendezaizai/Data_Imputation/imputation/results/business/ablation/business_with_retrieved_tuples_top20.jsonl', 'r') as f:
    for line in f:
        line = json.loads(line)
        tuple_id = int(line['tuple_id'])
        processed_tuples.append(tuple_id)

print(len(processed_tuples))

In [ ]:
def convert_to_table(serialized_tuple):
    # 分割标题和数据
    caption_split = serialized_tuple.split(' attribute ')
    title = caption_split[0].split(']: ')[1].strip()

    # 提取属性和值
    attributes = caption_split[1:]

    headers = []
    values = []
    sign = 0
    

    for attribute in attributes:
        attribute_value_split = attribute.split(' value ')
        attribute_name = attribute_value_split[0].strip()
        value = attribute_value_split[1].split(' attribute ')[0].strip()  # 分割可能的下一个属性
        
        if sign == 1 and len(attribute_name) > 10:
            attribute_name = attribute_name[:10]
        headers.append(attribute_name)
        
        values.append(value)

    # 构建表格
    table = 'caption: ' + title + '\n|' + ' | '.join(headers) + ' |\n|' + ' | '.join(values) + ' |'
    return table


In [ ]:
import json
import jsonlines
import pandas as pd
import ast

tuples = {}
template = '''Based on the retrieved tabular data, what's the most likely value for the [TO-FILL] cell in the table below? Please respond using JSON: {answer_format}, the key is attribute name of each [TO-FILL], value is the predicted value for each [TO-FILL].\n'''


'''
with open('/Users/yichendezaizai/Data_Imputation/data/business/annotated_data/folds.json', 'r') as f:
    folds = json.load(f)
    test_qids = folds['test']
'''

with open('/Users/yichendezaizai/Data_Imputation/imputation/results/business/ablation/test_ids.json', 'r') as f:
    test_qids = json.load(f)

missing_columns = ['CITY', 'ZIP CODE']


with open('/Users/yichendezaizai/Data_Imputation/data/business/annotated_data/queries.tsv', 'r') as f:
    for line in f:
        line = line.strip()
        qid = int(line[:line.index('\t')])
        query = line[line.index('\t')+1:]
        
        if qid not in test_qids or qid in processed_tuples:
            continue
        #if qid not in error_query_ids:
        #    continue
        
        query = convert_to_table(query)
        query = query.replace('N/A', '[TO-FILL]')

    
        answer_format = '''{CITY: "", ZIP CODE: ""}'''
        missing_pos = []
        ground_truth = []
        input_data = template + '\n' + query + '\n'
        
        input_data = input_data.format(answer_format=answer_format)

        # Adding retrieved tables
        input_data += 'Retrieved Tables:\n'
        retrieved_tables = topK_results[qid]
        for rank, docid in enumerate(retrieved_tables):
            input_data += 'Table ' + str(rank+1) + ': ' + convert_to_table(collection[docid]) + '\n\n'


        print("---------------------------------------------------")
        print(f"Input: \n{input_data}")

        output = chat(input_data, model="gpt-3.5-turbo", temperature=0.3)

        print(f"Output: \n{output}")
            
        #imputed_data = ast.literal_eval(output)
        
        fout = jsonlines.open('./results/business/ablation/business_with_retrieved_tuples_top20.jsonl', 'a')
        fout.write({'tuple_id':qid, 'input': input_data, 'output':output})
        fout.close()



### Calculate scores

In [ ]:
with open('/Users/yichendezaizai/Data_Imputation/data/business/annotated_data/folds.json', 'r') as f:
    folds = json.load(f)
    test_qids = folds['test']
test_qids = sorted(test_qids)

In [ ]:
fout =  open('./results/business/GPT_business_with_retrieved_tuples_by_monoT5_new.jsonl', 'w')
with open('./results/business/GPT_business_with_retrieved_tuples_by_monoT5.jsonl', 'r') as f:
    for i, line in enumerate(f):
        line = json.loads(line)
        line['tuple_id'] = test_qids[i]
        fout.write(json.dumps(line) + '\n')
fout.close()

In [ ]:
import json
import jsonlines
import pandas as pd
import ast

tuples = {}

error_query_ids = []

'''
with open('/Users/yichendezaizai/Data_Imputation/data/business/annotated_data/folds.json', 'r') as f:
    folds = json.load(f)
test_qids = folds['test']
'''
with open('/Users/yichendezaizai/Data_Imputation/imputation/results/business/ablation/test_ids.json', 'r') as f:
    test_qids = json.load(f)
#test_qids = folds['test']

imputed_record = {}
with open('./results/business/ablation/business_with_retrieved_tuples_top20.jsonl', 'r') as f:
    for line in f:
        line = json.loads(line)
        tuple_id = int(line['tuple_id'])
        imputed_record[line['tuple_id']] = ast.literal_eval(line['output'])


ground_truth = {}
with jsonlines.open('/Users/yichendezaizai/Data_Imputation/data/business/annotated_data/answers.jsonl', 'r') as f:
    for line in f:
        ground_truth[line['query_id']] = line['answers']


columns_to_keep = ['ACCOUNT NUMBER', 'SITE NUMBER', 'NAME', 'CITY', 'STATE', 'ZIP CODE', 'LICENSE DESCRIPTION', 'APPLICATION TYPE']

df = pd.read_csv('/Users/yichendezaizai/Data_Imputation/data/business/Business_Licenses_-_Pharmaceutical_Representative.csv', usecols=columns_to_keep)


count, acc = 0, 0
tuple_total = 0
missing_columns = ['CITY', 'ZIP CODE']
for qid, row in df.iterrows():
    if qid not in test_qids:
        continue

    tuple_total += 1
            
    imputed_data = imputed_record[qid]

    correct_values = {}
    for key, value in ground_truth[qid].items():
        correct_values[key] = []
        for vv in value:
            correct_values[key].append(vv.lower().replace('(','').replace(')',''))

    for i, col in enumerate(missing_columns):
        cell_value = str(imputed_data[col]).lower().replace('(','').replace(')','')
        try:
            cell_value = str(int(float(cell_value)))
        except:
            pass

        if cell_value in correct_values[col]:
            acc += 1
        else:
            error_query_ids.append(qid)
            print("---------------------------------------------------")
            print(f"tuple_id: {qid}")
            print(cell_value)
            print(correct_values[col])

        count += 1

print(f"Accuracy: {acc/count}")
print(acc, count)
print(f"tuple_total: {tuple_total}")